In [ ]:
import json
import re
import pandas as pd
import pyLDAvis

try:
    import tomotopy as tp
except Exception as e:
    print("Error importing tomotopy. Please install it using 'pip install tomotopy'")

from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from datetime import datetime
from nltk.tokenize import RegexpTokenizer
import numpy as np
import statistics
tokenizer = RegexpTokenizer(r'\w+')
import nltk
nltk.downloader.download('vader_lexicon')
nltk.download('stopwords')

def read_json(fname):
    with open(fname) as json_file:
        data = json.load(json_file)

    return data


def run_topic_models(df,  msg_sender, condition):
    try:
        # Load the preprocessed corpus
        corpus = tp.utils.Corpus.load('./data/corpus/mtype={}_cond={}.cps'.format( msg_sender, condition))
    except:
        stemmer = nltk.SnowballStemmer('english').stem
        english_stops = set(stemmer(w) for w in stopwords.words('english'))
        pat = re.compile('^[a-z]{2,}$')
        corpus = tp.utils.Corpus(
            tokenizer=tp.utils.SimpleTokenizer(stemmer),
            stopwords=lambda x: x in english_stops or not pat.match(x)
        )

        try:
            if condition == "all":
                conversations = df
            else:
                conversations = df[df['agent_cond'] == condition]

            if  msg_sender == "ai":
                conversations = conversations['AppendedAIMessages']
            elif  msg_sender == "human":
                conversations = conversations['AppendedHumanMessages']
            else:
                print("Error")
                return

        except:
            return

        corpus.process(conversation.lower() for conversation in conversations.values)
        corpus.save('./data/corpus/mtype={}_cond={}.cps'.format( msg_sender, condition))

    # Set seed for reproducibility
    
    mdl = tp.LDAModel(min_df=5, rm_top=5, k=15, corpus=corpus, seed=0)
    mdl.train(0)

    write_file = open("./topicmodel_logs/logs_mtype={}_condition={}.txt".format( msg_sender, condition), "w+")

    print('Num docs:{}, Num Vocabs:{}, Total Words:{}'.format(
        len(mdl.docs), len(mdl.used_vocabs), mdl.num_words
    ), file=write_file)
    print('Removed Top words: ', *mdl.removed_top_words, file=write_file)

    # Let's train the model
    for i in range(0, 1000, 20):
        print('Iteration: {:04}, LL per word: {:.4}'.format(i, mdl.ll_per_word), file=write_file)
        mdl.train(100)
    print('Iteration: {:04}, LL per word: {:.4}'.format(1000, mdl.ll_per_word), file=write_file)

    mdl.summary()

    topic_term_dists = np.stack([mdl.get_topic_word_dist(k) for k in range(mdl.k)])
    doc_topic_dists = np.stack([doc.get_topic_dist() for doc in mdl.docs])
    doc_topic_dists /= doc_topic_dists.sum(axis=1, keepdims=True)
    doc_lengths = np.array([len(doc.words) for doc in mdl.docs])
    vocab = list(mdl.used_vocabs)
    term_frequency = mdl.used_vocab_freq

    if len(vocab) < 10:
        print("Not enough conversations. Terminating...", file=write_file)
        return

    prepared_data = pyLDAvis.prepare(
        topic_term_dists,
        doc_topic_dists,
        doc_lengths,
        vocab,
        term_frequency,
        sort_topics=False,
        mds='mmds'
    )

    fname = './model_visuals/msg_sender={}_condition={}.html'.format( msg_sender, condition)

    pyLDAvis.save_html(prepared_data, fname)
    # return mdl_words  # Return top 10 words of each topic


def process():
    df = pd.read_csv('./data/clean_combined_data.csv')
    df = df.loc[df['interact_cond'] == 'yes']


    chatlogs = pd.read_csv('./data/chatlogs.csv')
    messages = pd.read_csv('./data/messages.csv')

    data = {
        'human': {'human_cond_processed': [], 'ai_cond_processed': [], 'human_cond_wait': [],
                  'ai_cond_wait': [], 'all_len': [], 'all_len_d': {}, 'human_cond_formatted': {},
                  'all_formatted': {}, 'ai_cond_formatted': {}},
        'ai': {'human_cond_processed': [], 'ai_cond_processed': [], 'human_cond_wait': [],
               'ai_cond_wait': [], 'all_len': [], 'all_len_d': {}, 'human_cond_formatted': {},
               'all_formatted': {}, 'ai_cond_formatted': {}}}

    def get_appended_human_messages(row):
        human = messages[(messages['worker_id'] == row['worker_id']) & (~pd.isna(messages['user_id']))]['content'].values.tolist()
        human = [h for h in human if type(h) == str]

        #return "Hidden"
        return " ".join(human)

    def get_appended_ai_messages(row):
        ai = messages[(messages['worker_id'] == row['worker_id']) & (pd.isna(messages['user_id']))]['content'].values.tolist()
        ai = [h for h in ai if type(h) == str]

        #return "Hidden"
        return " ".join(ai)

    def get_chatlog(row):
        #return "Hidden"
        return chatlogs[chatlogs['worker_id'] == row['worker_id']]['content'].values[0]

    def get_human_message_count(row):
        human = messages[(messages['worker_id'] == row['worker_id']) & (~pd.isna(messages['user_id']))][
            'content'].values.tolist()
        return len([h for h in human if type(h) == str])

    def get_ai_message_count(row):
        ai = messages[(messages['worker_id'] == row['worker_id']) & (pd.isna(messages['user_id']))][
            'content'].values.tolist()
        return len([h for h in ai if type(h) == str])

    def get_human_word_count(row):
        human = messages[(messages['worker_id'] == row['worker_id']) & (~pd.isna(messages['user_id']))][
            'content'].values.tolist()
        human = [len(h.split(" ")) for h in human if type(h) == str]
        return json.dumps(human)
        

    def get_ai_word_count(row):
        ai = messages[(messages['worker_id'] == row['worker_id']) & (pd.isna(messages['user_id']))][
            'content'].values.tolist()
        ai = [len(h.split(" ")) for h in ai if type(h) == str]
        return json.dumps(ai)
    
    def get_sum_human_char_count(row):
        human = messages[(messages['worker_id'] == row['worker_id']) & (~pd.isna(messages['user_id']))][
            'content'].values.tolist()
        human = sum([len(h) for h in human if type(h) == str])
        return json.dumps(human)
        

    def get_sum_ai_char_count(row):
        ai = messages[(messages['worker_id'] == row['worker_id']) & (pd.isna(messages['user_id']))][
            'content'].values.tolist()
        ai = sum([len(h) for h in ai if type(h) == str])
        return json.dumps(ai)

    def get_avg_human_word_count(row):
        human = messages[(messages['worker_id'] == row['worker_id']) & (~pd.isna(messages['user_id']))][
            'content'].values.tolist()
        human = [len(h.split(" ")) for h in human if type(h) == str]
        return statistics.mean(human)

    def get_avg_ai_word_count(row):
        ai = messages[(messages['worker_id'] == row['worker_id']) & (pd.isna(messages['user_id']))][
            'content'].values.tolist()
        ai = [len(h.split(" ")) for h in ai if type(h) == str]
        return statistics.mean(ai)

    def get_response_times(row):
        msgs = messages[(messages['worker_id'] == row['worker_id'])]
        response_times = []

        last_ai_timestamp = None
        last_ai_message = None
        ai_started = False
        last_message_is_human = False
        for index, msg in msgs.iterrows():
            if not ai_started and not pd.isna(msg['user_id']):  # Human sent a message before AI did
                continue
            elif pd.isna(msg['user_id']):
                ai_started = True

            if pd.isna(msg['user_id']):  # AI sends message
                last_ai_timestamp = datetime.strptime(msg['date_added'], "%Y-%m-%d %H:%M:%S.%f+00")
                last_ai_message = msg['content']
                last_message_is_human = False
            else:  # Human sends message
                if not last_message_is_human:
                    # Removing the time elapsed for 'Writing...' notification, as well as the estimated typing/reading time
                    response_time = (datetime.strptime(msg['date_added'], "%Y-%m-%d %H:%M:%S.%f+00") -
                                     last_ai_timestamp).total_seconds()
                    
                    response_time -= len(last_ai_message.split(" ")) * 0.6  # 0.6 second for each word
                    response_times.append(response_time)

                last_message_is_human = True

        return json.dumps(response_times)

    def get_avg_response_time(row):
        return statistics.mean(json.loads(get_response_times(row)))

    sia = SentimentIntensityAnalyzer()
    def get_human_compound_sentiments(row):
        msgs_human = messages[(messages['worker_id'] == row['worker_id']) & (~pd.isna(messages['user_id']))][
            'content'].values.tolist()
        return json.dumps([sia.polarity_scores(message)['compound'] for message in msgs_human if type(message) == str])

    def get_ai_compound_sentiments(row):
        msgs_ai = messages[(messages['worker_id'] == row['worker_id']) & (pd.isna(messages['user_id']))][
            'content'].values.tolist()
        return json.dumps([sia.polarity_scores(message)['compound'] for message in msgs_ai if type(message) == str])

    def get_avg_human_compound_sentiment(row):
        msgs_human = messages[(messages['worker_id'] == row['worker_id']) & (~pd.isna(messages['user_id']))][
            'content'].values.tolist()
        return statistics.mean([sia.polarity_scores(message)['compound'] for message in msgs_human if type(message) == str])

    def get_avg_ai_compound_sentiment(row):
        msgs_ai = messages[(messages['worker_id'] == row['worker_id']) & (pd.isna(messages['user_id']))][
            'content'].values.tolist()
        return statistics.mean([sia.polarity_scores(message)['compound'] for message in msgs_ai if type(message) == str])

    df['AppendedHumanMessages'] = df.apply(get_appended_human_messages, axis=1)
    df['AppendedAIMessages'] = df.apply(get_appended_ai_messages, axis=1)
    df['FullChatlog'] = df.apply(get_chatlog, axis=1)
    df['HumanMessageCount'] = df.apply(get_human_message_count, axis=1)
    df['AIMessageCount'] = df.apply(get_ai_message_count, axis=1)
    df['HumanWordCounts'] = df.apply(get_human_word_count, axis=1)
    df['AIWordCounts'] = df.apply(get_ai_word_count, axis=1)
    df['AvgHumanWordCount'] = df.apply(get_avg_human_word_count, axis=1)
    df['AvgAIWordCount'] = df.apply(get_avg_ai_word_count, axis=1)
    df['SumHumanCharCount'] = df.apply(get_sum_human_char_count, axis=1)
    df['SumAICharCount'] = df.apply(get_sum_ai_char_count, axis=1)
    df['ResponseTimes'] = df.apply(get_response_times, axis=1)
    df['AICompoundSentiments'] = df.apply(get_ai_compound_sentiments, axis=1)
    df['HumanCompoundSentiments'] = df.apply(get_human_compound_sentiments, axis=1)
    df['AvgAICompoundSentiment'] = df.apply(get_avg_ai_compound_sentiment, axis=1)
    df['AvgHumanCompoundSentiment'] = df.apply(get_avg_human_compound_sentiment, axis=1)
    df['AvgResponseTime'] = df.apply(get_avg_response_time, axis=1)

    df.to_csv('./data/clean_detailed_int_data.csv')


### Generate Conversation Features

In [ ]:
process()

### Read Generated File Containing Conversation Features

In [ ]:
df = pd.read_csv('./data/clean_detailed_int_data.csv')

### Run Topic Models

In [ ]:
for msg_sender in ['human', 'ai']:
    for condition in ['human', 'ai']:
        run_topic_models(df, msg_sender=msg_sender, condition=condition)

# Code Below Are For Pairwise Analyses (e.g., sentiment similarity)

## Run 'python -m spacy download en_core_web_lg' to download the large English model for Spacy

In [ ]:
############## IMPORTS ##############

from sklearn.linear_model import LinearRegression
import numpy as np
from collections import Counter
import pandas as pd
import nltk
import statistics
from rpy2.robjects.packages import importr
import rpy2.robjects as R
from rpy2.robjects import pandas2ri
pandas2ri.activate()
from rpy2.robjects.packages import importr

utils = importr('utils')
utils.chooseCRANmirror(ind=1) # select the first mirror in the list
utils.install_packages('effsize')
effsize = importr('effsize')

import csv
import spacy
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import RegexpTokenizer

import spacy.cli

spacy.cli.download("en_core_web_lg")
nlp = spacy.load("en_core_web_lg")

base = importr('base')
stats = importr("stats")

nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

# Tagset: https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html

In [ ]:
############## READ THE DATA ##############

results = {"human": [], "ai": [], "all": []}

sender_type = "human" # ai, human, all
label_type = "willing_friend"  # willing_friend, willing_romantic, wtp

# Dataset
data_all = df[['FullChatlog', 'AppendedHumanMessages', 'AppendedAIMessages', 'willing_friend', 'willing_romantic', 'wtp', 'agent_cond', 'worker_id', 'belief']]

all_messages = {'ai': [], 'human': []}

In [ ]:
############## CONSTRUCT LSM INSTANCE ##############

class LinguisticStyleMatching:
    data = None
    sia = None
    tokenizer = None

    def __init__(self):
        self.data = data_all[['AppendedHumanMessages', 'AppendedAIMessages', 'willing_friend', 'willing_romantic', 'wtp', 'agent_cond', 'worker_id', 'belief']]
        self.sia = SentimentIntensityAnalyzer()
        self.tokenizer = RegexpTokenizer(r'\w+')

    # Return each AI and human message separately, to compare later
    def get_paired_messages(self):
        data = pd.read_csv('./data/messages.csv')
        curr_worker = None

        all_pairs = {}
        curr_message_pairs = {'human-ai': [], 'ai-human': []}
        last_message = {'sender_type': '', 'content': ''}

        for index, message in data.iterrows():
            curr_worker = message['worker_id'] if curr_worker is None else curr_worker  # First Iteration

            if curr_worker != message['worker_id']: # Room changed, save
                all_pairs[curr_worker] = curr_message_pairs
                curr_worker = message['worker_id']

                # Reset
                curr_message_pairs = {'human-ai': [], 'ai-human': []}
                last_message = {'sender_type': '', 'content': ''}

            # Keep adding to curr_messages
            if message['username'] == 'Jessie': # AI message
                if last_message['sender_type'] == 'ai':  # Take only the first message, in case there are consecutive ones
                    continue

                if last_message['sender_type'] == 'human':  # Second message, Save the pair
                    curr_message_pairs['human-ai'].append([last_message['content'], message['content']])

                last_message = {'sender_type': 'ai', 'content': message['content']}
            else:
                if last_message['sender_type'] == 'human':
                    continue

                if last_message['sender_type'] == 'ai':  # Not None
                    curr_message_pairs['ai-human'].append([last_message['content'], message['content']])

                last_message = {'sender_type': 'human', 'content': message['content']}

            if index == data.shape[0] - 1:
                all_pairs[curr_worker] = curr_message_pairs


        return all_pairs


    # Normalize the data to interval [0, 1], before calculating similarity
    # zi = (xi – min(x)) / (max(x) – min(x))
    def normalize_val(self, num):
        return (num + 1) / 2


    # Calculate average turn-by-turn similarities and add it to self.data
    def calculate_turn_similarities(self):

        # Create Columns
        for sim_type in ['Spacy', 'TurnLength', 'Valence']:
            self.data['HumanAISim{}'.format(sim_type)] = None
            self.data['AIHumanSim{}'.format(sim_type)] = None
            self.data['AvgSim{}'.format(sim_type)] = None

        paired_messages = self.get_paired_messages()

        for worker_id, pairs in paired_messages.items():  # All subjects
            sim = {'Spacy': {}, 'TurnLength': {}, 'Valence': {}}
            for pair_type, messages in pairs.items():  # Iterate Both Pair Types, i.e., Human-AI or AI-Human
                similarities = {'Spacy': [], 'TurnLength': [], 'Valence': []}
                for pair in messages:  # Iterate each pair
                    try:
                        #### Calculate Spacy Similarity ####
                        spacy_similarity = nlp(str(pair[0])).similarity(nlp(str(pair[1])))
                        similarities['Spacy'].append(100 * spacy_similarity)

                        #### Calculate Turn Length Similarity ####
                        turn_similarity = self.calculate_sim(len(self.tokenizer.tokenize(str(pair[0]))), len(self.tokenizer.tokenize(str(pair[1]))))
                        similarities['TurnLength'].append(turn_similarity)

                        #### Calculate Valence Similarity ####
                        # Normalize the data to interval [0, 1], before calculating similarity
                        polarity_1 = self.normalize_val(self.sia.polarity_scores(str(pair[0]))['compound'])
                        polarity_2 = self.normalize_val(self.sia.polarity_scores(str(pair[1]))['compound'])
                        valence_similarity = self.calculate_sim(polarity_1, polarity_2)
                        similarities['Valence'].append(valence_similarity)

                        #print(" -- Spacy Similarity: ", spacy_similarity, " -- Turn Length Similarity: ", turn_similarity, " -- Valence Similarity: ", valence_similarity)
                    except Exception as e:
                        print(e)

                # Calculate mean and std. deviation between pairs
                for sim_type in similarities.keys():
                    sim[sim_type][pair_type] = {'mean': statistics.mean(similarities[sim_type]), 'sd': statistics.stdev(similarities[sim_type])}

            for sim_type in sim.keys():
                self.data.loc[self.data['worker_id'] == worker_id, 'HumanAISim{}'.format(sim_type)] = sim[sim_type]['human-ai']['mean']
                self.data.loc[self.data['worker_id'] == worker_id, 'AIHumanSim{}'.format(sim_type)] = sim[sim_type]['ai-human']['mean']
                self.data.loc[self.data['worker_id'] == worker_id, 'AvgSim{}'.format(sim_type)] = statistics.mean([sim[sim_type]['ai-human']['mean'], sim[sim_type]['human-ai']['mean']])



    def tokenize_raw_data(self):
        def tokenize(conversation):
            return nltk.pos_tag(nltk.word_tokenize(conversation))

        self.data.loc[:, 'HumanTags'] = self.data.loc[:, 'AppendedHumanMessages'].apply(tokenize)
        self.data.loc[:, 'AITags'] = self.data.loc[:, 'AppendedAIMessages'].apply(tokenize)

    def count_tokens(self):
        def count_tokens(tokens):
            return dict(Counter(token[1] for token in tokens))

        self.data.loc[:, 'HumanTokenCounts'] = self.data.loc[:, 'HumanTags'].apply(count_tokens)
        self.data.loc[:, 'AITokenCounts'] = self.data.loc[:, 'AITags'].apply(count_tokens)

    def calculate_style_matching(self):
        def calculate(human_tags, ai_tags):
            # Ignore Punctuations
            if "." in human_tags.keys():
                human_tags.pop(".")

            if "." in ai_tags.keys():
                ai_tags.pop(".")

            human_total_tags = sum(human_tags.values())
            ai_total_tags = sum(ai_tags.values())
            matchings = {}
            for k in self.get_all_token_names(): # Calculate only function words
                human_prop = human_tags.get(k, 0) / human_total_tags
                ai_prop = ai_tags.get(k, 0) / ai_total_tags
                matchings[k] = self.calculate_sim(human_prop, ai_prop)

            return matchings

        self.data['StyleMatching'] = self.data.apply(lambda x: calculate(x.HumanTokenCounts, x.AITokenCounts), axis=1)

    def get_all_token_names(self):
        return ['CC', 'DT', 'PRP', 'PRP$', 'WP', 'IN']

    def get_all_tokens(self):
        tokens = self.get_all_token_names()

        def get_token(row): # Return tokens
            return { token: row.get(token, 0) for token in tokens }

        return self.data.loc[:, 'StyleMatching'].apply(get_token)

    # Mean of all common tokens
    def get_lsms(self):
        tokens = self.get_all_tokens()
        token_names = self.get_all_token_names()

        def get_avg(row):
            avg = []
            for token in token_names:
                avg.append(row[token])

            return statistics.mean(avg)

        self.data['AvgSimStyleMatching'] = self.data.loc[:, 'StyleMatching'].apply(get_avg)
        return self.data['AvgSimStyleMatching']


    # Mean LSM among all conversations
    def get_mean_lsm(self):
        tokens = self.get_all_token_names()
        token_avgs = { token : [] for token in tokens }
        for index, row in self.data.iterrows():
            for token in tokens:
                token_avgs[token].append(row['StyleMatching'][token])

        return { k: {'Mean': sum(v) / len(v), 'Std. Dev': statistics.stdev(v)} for k, v in token_avgs.items() }

    # Returns the similarity between two numbers, i.e., similarity = 1 − (|num1 − num2|/(num1 + num2))
    def calculate_sim(self, num1, num2):
        if num1 + num2 == 0:
            return 100
        return 100 * (1 - (abs(num1 - num2) / (num1 + num2)))



### Setup For Style Matching

In [ ]:
lsm = LinguisticStyleMatching()
lsm.tokenize_raw_data()
lsm.count_tokens()
lsm.calculate_style_matching()
lsm.get_mean_lsm()
lsm.get_all_tokens()
lsms = lsm.get_lsms()

In [ ]:
lsm.get_mean_lsm()

In [ ]:

nlp = spacy.load("en_core_web_lg")
lsm.calculate_turn_similarities()  # Calculate turn-by-turn similarities

### LINEAR REGRESSION ANALYSIS ON LSM TOKEN & TURN LENGTH & SENTIMENT & SEMANTIC SIMILARITIES

In [ ]:
dfn = pd.DataFrame({'lsm': lsm.data['AvgSimStyleMatching'],
                    'semantic_sim': pd.to_numeric(lsm.data['AvgSimSpacy']),
                    'sentiment_sim': pd.to_numeric(lsm.data['AvgSimValence']),
                    'turn_length_sim': pd.to_numeric(lsm.data['AvgSimTurnLength']),

                    'message_count': df['HumanMessageCount'],
                    'avg_word_count': df['AvgHumanWordCount'],
                    'avg_response_time': df['AvgResponseTime'],
                    'avg_sentiment': df['AvgHumanCompoundSentiment'],

                    'willing_friend': lsm.data['willing_friend'],
                    'willing_romantic': lsm.data['willing_romantic'],
                    'wtp': lsm.data['wtp'], 'agent_cond': lsm.data['agent_cond']})

df_human = dfn[dfn['agent_cond'] == 'human']
df_ai = dfn[dfn['agent_cond'] == 'ai'] # AI Condition

dfs = {'human': df_human, 'ai': df_ai, 'all': dfn}
for condition in ['all']: # 'human', 'ai'
    for dv in ['willing_friend', 'willing_romantic', 'wtp']:
        for similarity_type in ['lsm', 'semantic_sim', 'sentiment_sim', 'turn_length_sim', 'message_count', 'avg_word_count', 'avg_response_time', 'avg_sentiment']:
            print("******* {} Condition -- Does {} predict {}? *******".format(condition, similarity_type, dv))
            lm = stats.lm('{} ~ {}'.format(dv, similarity_type), data=dfs[condition]) # For logistic regression: R.r.glm
            print(str(base.summary(lm)).split('Coefficients:')[1])

            print("M: ", dfs[condition]['lsm'].mean())
            print("SD: ", dfs[condition]['lsm'].std())
            print("*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*")

In [ ]:
lsm.data[['worker_id', 'AvgSimSpacy', 'AvgSimTurnLength', 'AvgSimValence', 'AvgSimStyleMatching']]

### Save the data

In [ ]:
# Merge with our original dataset
df = df.merge(lsm.data[['worker_id', 'AvgSimSpacy', 'AvgSimTurnLength', 'AvgSimValence', 'AvgSimStyleMatching']], on='worker_id')
df.to_csv("./data/clean_detailed_int_data.csv")  ### Save similarity data

## t-tests

In [ ]:
lsm_df = pd.DataFrame({'AvgSimSpacy': pd.to_numeric(lsm.data['AvgSimSpacy']),
                   'HumanAISimSpacy': pd.to_numeric(lsm.data['HumanAISimSpacy']), 
                   'AIHumanSimSpacy': pd.to_numeric(lsm.data['AIHumanSimSpacy']), 

                   'AvgSimValence': pd.to_numeric(lsm.data['AvgSimValence']), 
                   'HumanAISimValence': pd.to_numeric(lsm.data['HumanAISimValence']), 
                   'AIHumanSimValence': pd.to_numeric(lsm.data['AIHumanSimValence']), 

                   'AvgSimTurnLength': pd.to_numeric(lsm.data['AvgSimTurnLength']), 
                   'HumanAISimTurnLength': pd.to_numeric(lsm.data['HumanAISimTurnLength']), 
                   'AIHumanSimTurnLength': pd.to_numeric(lsm.data['AIHumanSimTurnLength']),

                   'StyleMatching': pd.to_numeric(lsm.data['AvgSimStyleMatching']),
                   
                   'willing_friend': lsm.data['willing_friend'],
                   'willing_romantic': lsm.data['willing_romantic'],
                   'wtp': lsm.data['wtp'], 'agent_cond': lsm.data['agent_cond'],
                   'worker_id': lsm.data['worker_id'], 'belief': lsm.data['belief']})
                          
df_human = lsm_df[lsm_df['agent_cond'] == 'human'] # Human Condition
df_human = df_human[df_human['belief'] == 1]
df_ai = lsm_df[lsm_df['agent_cond'] == 'ai'] # AI Condition

dfs = {'human': df_human, 'ai': df_ai, 'all': df}
for compare_with in ['human']:
    for pair_type in ['Avg']:
        for similarity_type in ['StyleMatching', 'Spacy', 'TurnLength', 'Valence']:
            print("*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-* Pair Type: {} - Human vs. AI Condition: {} *-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*".format(pair_type, similarity_type))
            vt = stats.var_test(R.FloatVector(dfs[compare_with]['{}Sim{}'.format(pair_type, similarity_type) if similarity_type != 'StyleMatching' else 'StyleMatching']),
            R.FloatVector(dfs['ai']['{}Sim{}'.format(pair_type, similarity_type) if similarity_type != 'StyleMatching' else 'StyleMatching']))


            tt = stats.t_test(R.FloatVector(dfs[compare_with]['{}Sim{}'.format(pair_type, similarity_type) if similarity_type != 'StyleMatching' else 'StyleMatching']),
            R.FloatVector(dfs['ai']['{}Sim{}'.format(pair_type, similarity_type) if similarity_type != 'StyleMatching' else 'StyleMatching']),
                **{'var.equal': float(vt.rx('p.value')[0][0])>0.05, 'paired': False})

            print(tt)

            sig = "None"
            if tt.rx('p.value')[0][0] < 0.001:
                sig = "***"
            elif tt.rx('p.value')[0][0] < 0.01:
                sig = "**"
            elif tt.rx('p.value')[0][0] < 0.05:
                sig = "*"
            elif tt.rx('p.value')[0][0] < 0.1:
                sig = "."

            print("Significance: ", sig)

            #d = effsize.cohen_d(R.FloatVector(dfs['human']['{}Sim{}'.format(pair_type, similarity_type)]),
            #R.FloatVector(dfs['ai']['{}Sim{}'.format(pair_type, similarity_type)]))

            #print(d)
            print("*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*")

In [ ]:
############## t-test between 'HumanAISim' vs 'AIHumanSim' ##############

lsm_df = pd.DataFrame({'AvgSimSpacy': pd.to_numeric(lsm.data['AvgSimSpacy']),
                   'HumanAISimSpacy': pd.to_numeric(lsm.data['HumanAISimSpacy']),
                   'AIHumanSimSpacy': pd.to_numeric(lsm.data['AIHumanSimSpacy']),

                   'AvgSimValence': pd.to_numeric(lsm.data['AvgSimValence']),
                   'HumanAISimValence': pd.to_numeric(lsm.data['HumanAISimValence']),
                   'AIHumanSimValence': pd.to_numeric(lsm.data['AIHumanSimValence']),

                   'AvgSimTurnLength': pd.to_numeric(lsm.data['AvgSimTurnLength']),
                   'HumanAISimTurnLength': pd.to_numeric(lsm.data['HumanAISimTurnLength']),
                   'AIHumanSimTurnLength': pd.to_numeric(lsm.data['AIHumanSimTurnLength']),

                   'willing_friend': lsm.data['willing_friend'],
                   'willing_romantic': lsm.data['willing_romantic'],
                   'wtp': lsm.data['wtp'], 'agent_cond': lsm.data['agent_cond'], 'belief': lsm.data['belief']})

df_human = lsm_df[lsm_df['agent_cond'] == 'human'] # Human Condition
df_ai = lsm_df[lsm_df['agent_cond'] == 'ai'] # AI Condition

dfs = {'human': df_human, 'ai': df_ai, 'all': lsm_df}
for condition in ['human', 'ai', 'all']:
    for dv_type in ['Spacy', 'TurnLength', 'Valence']:
        print("*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-* {} -- {} *-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*".format(dv_type, condition))
        vt = stats.var_test(R.FloatVector(dfs[condition]['HumanAISim{}'.format(dv_type)]),
        R.FloatVector(dfs[condition]['AIHumanSim{}'.format(dv_type)]))


        tt = stats.t_test(R.FloatVector(dfs[condition]['HumanAISim{}'.format(dv_type)]),
        R.FloatVector(dfs[condition]['AIHumanSim{}'.format(dv_type)]),
            **{'var.equal': float(vt.rx('p.value')[0][0])>0.05, 'paired': False})

        print(tt)

        sig = "None"
        if tt.rx('p.value')[0][0] < 0.001:
            sig = "***"
        elif tt.rx('p.value')[0][0] < 0.01:
            sig = "**"
        elif tt.rx('p.value')[0][0] < 0.05:
            sig = "*"
        elif tt.rx('p.value')[0][0] < 0.1:
            sig = "."

        print("Significance: ", sig)

        d = effsize.cohen_d(R.FloatVector(dfs[condition]['HumanAISim{}'.format(dv_type)]),
        R.FloatVector(dfs[condition]['AIHumanSim{}'.format(dv_type)]))
        print(d)

        print("Spacy Human-AI M = ", dfs[condition]['{}Spacy'.format("HumanAISim")].mean())
        print("Spacy Human-AI SD = ", dfs[condition]['{}Spacy'.format("HumanAISim")].std())

        print("Spacy AI-Human M = ", dfs[condition]['{}Spacy'.format("AIHumanSim")].mean())
        print("Spacy AI-Human SD = ", dfs[condition]['{}Spacy'.format("AIHumanSim")].std())

        print("Turn Length Human-AI M = ", dfs[condition]['{}TurnLength'.format("HumanAISim")].mean())
        print("Turn Length Human-AI SD = ", dfs[condition]['{}TurnLength'.format("HumanAISim")].std())

        print("Turn Length AI-Human M = ", dfs[condition]['{}TurnLength'.format("AIHumanSim")].mean())
        print("Turn Length AI-Human SD = ", dfs[condition]['{}TurnLength'.format("AIHumanSim")].std())

        print("Sentiment Human-AI M = ", dfs[condition]['{}Valence'.format("HumanAISim")].mean())
        print("Sentiment Human-AI SD = ", dfs[condition]['{}Valence'.format("HumanAISim")].std())

        print("Sentiment AI-Human M = ", dfs[condition]['{}Valence'.format("AIHumanSim")].mean())
        print("Sentiment AI-Human SD = ", dfs[condition]['{}Valence'.format("AIHumanSim")].std())

        print("*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*")